# Prefactor value required to simulate a given SNR 

First we import the required libraries.

In [ ]:
import os
from os.path import isfile, expandvars
from sagsci.tools.utils import *

We need a background simulation to estimate the background counts in a given region of the field of view. Then we need to save the configuration of the background similation (which will also be the same configuration for the simulation of the source).

In [ ]:
# observation and target
obs_bkg = '../data/bkg_test_sim.fits'
target = {'ra': 83.6331, 'dec': 22.0145}

# configuration
sigma = 4                  # significance in gaussian sigmas
erange = [0.03, 150]       # energy in teraelectronvolt (TeV)
trange = [0, 1200]         # livetime in seconds (s)
radius = 0.2               # photometry region in degrees (deg)
spectral_index = -2.48     # slope of the power-law spectrum

# we need to add "radius" to the target dictionary 
target['rad'] = radius

We will also need the Instrument Response Function (IRF) file. If you have installed **ctools** then you can use the <code>$CTOOLS</code> environmental variable, or you can export the path where you installed the prod folder. 

Alternatively you will just have to pass the correct path to the irf file.

In [ ]:
# using $CTOOLS variable:
irf = expandvars('$CTOOLS/share/caldb/data/cta/prod3b-v2/bcf/South_z40_0.5h/irf_file.fits')
# if you have installed the caldb elswhere the you can pass the path:
#irf = '../../caldb/data/cta/prod3b-v2/bcf/South_z40_0.5h/irf_file.fits'

From the background simulation we can retrive the poiting.

In [ ]:
pointing = get_obs_pointing(filename=obs_bkg)
print(f'pointing = {pointing} deg')

### STEP 1 - Background counts

Now we count the background photons found within a region of <code>radius</code> centered on coordinates where we want to simulate the source.

In [ ]:
# remove existing files
offregionsfile = obs_bkg.replace('.fits', '.reg')
if isfile(offregionsfile):
    os.remove(offregionsfile)
    
# compute counts
bkg_counts_in_region = get_counts_in_region(filename=obs_bkg, target=target, pointing=pointing, trange=trange, erange=erange, off_regions=offregionsfile)

# print background counts
print(f'background counts = {bkg_counts_in_region} counts')

### STEP 2 - Prefactor given sigma and background counts

Given the configured sigma and the computed background counts, we can now find the prefactor required to simulate the source at the desired significance level.

In [ ]:
prefactor = get_prefactor_from_bkg_and_sigma(sigma=sigma, bkg=bkg_counts_in_region, target=target, 
                                             pointing=pointing, trange=trange, erange=erange, irf=irf, 
                                             spectral_index=spectral_index)
# print prefactor
print(f'prefactor = {prefactor} ph/cm2/s/MeV')

### Explanation of STEP 2

The function <code>get_prefactor_from_bkg_and_sigma</code> is a wrapper that executes the following procedure. First we compute the expected counts excess for a given <code>sigma</code> given the background counts.

In [ ]:
excess = get_excess_from_sigma_and_bkg(sigma=sigma, bkg=bkg_counts_in_region)
print(f'excess = {excess} counts')

Then we can check the inverse of the forumla to verify we obtain the same significance.

In [ ]:
snr = get_snr(signal=excess, bkg=bkg_counts_in_region)
print(f'snr = {snr} sigmas')

In order to obtain the flux from the excess counts, we need to compute the exposure (the effective area times the livetime) from the IRF.

In [ ]:
exposure = get_exposure_in_region(target=target, pointing=pointing, trange=trange, erange=erange, irf=irf, index=spectral_index)
print(f'exposure = {exposure} cm2/s')

The Flux is then found as the excess divided by the exposure.

In [ ]:
flux = excess / exposure 
print(f'flux = {flux} ph/cm2/s')

From the flux we can, given the energy range and a spectral index, compute the required prefactor. This value will be the amplitude of the spectrum to use when simulating a source at <code>sigma</code>.

In [ ]:
prefactor = get_prefactor(flux=flux, erange=erange, gamma=spectral_index, unit='TeV')
print(f'prefactor = {prefactor} ph/cm2/s/MeV')